In [1]:
from v3_pool import *

## Test constructors

In [2]:
Token("ETH",18)

Token(name='ETH', decimals=18)

In [3]:
GlobalState()

GlobalState(L=0.0, rP=0.0, tick=0, fg_x=0.0, fg_y=0.0, proto_x=0.0, proto_y=0.0)

In [4]:
TickState()

TickState(liq_net=0.0, liq_gross=0.0, f0_x=0.0, f0_y=0.0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)

In [5]:
PositionState()

PositionState(liq=0.0, fr_x=0.0, fr_y=0.0)

In [6]:
Pool("ETH", 18, "USDC", 6, bootstrap_rP=4000**0.5)

ETH-USDC pool - tick spacing 1
Token(name='ETH', decimals=18) Token(name='USDC', decimals=6)
GlobalState(L=0.0, rP=63.245553203367585, tick=82944, fg_x=0.0, fg_y=0.0, proto_x=0.0, proto_y=0.0)
real reserve X=0.0 Y=0.0


## Test static method formulas

Problem 1:  
A user has x = 2 ETH and wants to set up a liquidity position in an ETH/USDC pool. The current price of ETH is P =2000 USDC and target price range is from pa = 1500 to pb = 2500 USDC. How much USDC (y) do they need?

In [7]:
# 2-step: get L first
a,P,b = 1500,2000,2500
x = 2.0 
# First, calculate the liquidity of the top half of the range by using
# reserve x cover from current P to upper bound (pretend P is lower bound)
Lx = Pool.liq_x_only(x=x,rPa=P**0.5, rPb=b**0.5)
Lx

847.2135954999583

In [8]:
# 2-step: then get y . 
y = Pool.y_from_L_rP_rng(L=Lx,rP=P**0.5,rPa=a**0.5,rPb=b**0.5)
y

5076.102359479882

Problem 2:  
A user has x=2 ETH and y=4000 USDC, and wants to use pb =3000USDC perETH as the top of the price range. What is the bottom of the range (pa) that ensures the opened position uses the full amount of their funds?

In [9]:
# Bounds in 1-step
x,y  = 2.0, 4000
P = y/x
b= 3000

rPa = Pool.rPa_from_x_y_rP_rPb(x=x,y=y,rP=P**0.5,rPb=b**0.5)
rPa**2

1333.3333333333333

In [10]:
# Bounds in 2-steps
# compute L using the top part of range, given we know x 
Lx = Pool.liq_x_only(x,rPa=P**0.5,rPb=b**0.5)
# then get lower bound from L, price and y
rPa = Pool.rPa_from_L_rP_y(L=Lx,rP=P**0.5,y=y)
rPa**2

1333.3333333333337

Problem 3:  
Using the liquidity position created in Problem 2, what are asset balances when the price changes to P = 2500 USDC per ETH?

In [11]:
# compute L from price when position established, reserves and bounds
L = Pool.liq_from_x_y_rP_rng(x=x,y=y,rP=P**0.5, rPa=rPa, rPb=b**0.5)

x_new = Pool.x_from_L_rP_rng(L=L, rP=2500**0.5, rPa=rPa, rPb=b**0.5)

y_new = Pool.y_from_L_rP_rng(L=L, rP=2500**0.5, rPa=rPa, rPb=b**0.5)

x_new , y_new

(0.8493641204744684, 6572.9000439693455)

In [12]:
# or calc change of reserves based on change of price
dX =  Pool.dX_from_L_drP(L,rP_old=P**0.5,rP_new=2500**0.5)
print(f"{L=} {dX=} x_new={x + dX}")

L=487.41718030204123 dX=-1.1506358795255318 x_new=0.8493641204744682


In [13]:
dY =  Pool.dY_from_L_drP(L,rP_old=P**0.5,rP_new=2500**0.5)
print(f"{dY=} y_new={y + dY}")

dY=2572.9000439693473 y_new=6572.900043969347


-------------
-------------
-------------
## Test pool mechanics

### Pool Init and tick math

In [14]:
pool= Pool("ETH", 18, "USDC", 6, bootstrap_rP=4000**0.5,tick_spacing=30)
pool

ETH-USDC pool - tick spacing 30
Token(name='ETH', decimals=18) Token(name='USDC', decimals=6)
GlobalState(L=0.0, rP=63.245553203367585, tick=82920, fg_x=0.0, fg_y=0.0, proto_x=0.0, proto_y=0.0)
real reserve X=0.0 Y=0.0


In [15]:
assert(pool.tick_to_rP(pool.global_state.tick) <= pool.global_state.rP)

In [16]:
print(pool.rP_to_tick(pool.global_state.rP), pool.rP_to_possible_tick(pool.global_state.rP))
print(pool.rP_to_tick(pool.global_state.rP), pool.rP_to_possible_tick(pool.global_state.rP, left_to_right=True))

82944 82920
82944 82950


------------
------------
1. set_position()

In [17]:
pool= Pool("ETH", 18, "USDC", 6, bootstrap_rP=4000**0.5,tick_spacing=100)
pool

ETH-USDC pool - tick spacing 100
Token(name='ETH', decimals=18) Token(name='USDC', decimals=6)
GlobalState(L=0.0, rP=63.245553203367585, tick=82900, fg_x=0.0, fg_y=0.0, proto_x=0.0, proto_y=0.0)
real reserve X=0.0 Y=0.0


In [18]:

pool._set_position("123", 80000, 90000, 300)
pool.show()

GlobalState(L=300.0, rP=63.245553203367585, tick=82900, fg_x=0.0, fg_y=0.0, proto_x=0.0, proto_y=0.0)
real reserves X=0.0 Y=0.0
---active ticks---
tick '80000': TickState(liq_net=300.0, liq_gross=300.0, f0_x=0.0, f0_y=0.0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
tick '90000': TickState(liq_net=-300.0, liq_gross=300.0, f0_x=0, f0_y=0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
---positions---
poz '('123', 80000, 90000)': PositionState(liq=300, fr_x=0.0, fr_y=0.0)


In [19]:
pool._set_position("888", 90000,95000 , 200)
pool.show()

GlobalState(L=300.0, rP=63.245553203367585, tick=82900, fg_x=0.0, fg_y=0.0, proto_x=0.0, proto_y=0.0)
real reserves X=0.0 Y=0.0
---active ticks---
tick '80000': TickState(liq_net=300.0, liq_gross=300.0, f0_x=0.0, f0_y=0.0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
tick '90000': TickState(liq_net=-100.0, liq_gross=500.0, f0_x=0, f0_y=0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
tick '95000': TickState(liq_net=-200.0, liq_gross=200.0, f0_x=0, f0_y=0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
---positions---
poz '('123', 80000, 90000)': PositionState(liq=300, fr_x=0.0, fr_y=0.0)
poz '('888', 90000, 95000)': PositionState(liq=200, fr_x=0.0, fr_y=0.0)


In [20]:
pool._set_position("888", 70000, 95000, 400)
pool.show()

GlobalState(L=700.0, rP=63.245553203367585, tick=82900, fg_x=0.0, fg_y=0.0, proto_x=0.0, proto_y=0.0)
real reserves X=0.0 Y=0.0
---active ticks---
tick '80000': TickState(liq_net=300.0, liq_gross=300.0, f0_x=0.0, f0_y=0.0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
tick '90000': TickState(liq_net=-100.0, liq_gross=500.0, f0_x=0, f0_y=0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
tick '95000': TickState(liq_net=-600.0, liq_gross=600.0, f0_x=0, f0_y=0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
tick '70000': TickState(liq_net=400.0, liq_gross=400.0, f0_x=0.0, f0_y=0.0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
---positions---
poz '('123', 80000, 90000)': PositionState(liq=300, fr_x=0.0, fr_y=0.0)
poz '('888', 90000, 95000)': PositionState(liq=200, fr_x=0.0, fr_y=0.0)
poz '('888', 70000, 95000)': PositionState(liq=400, fr_x=0.0, fr_y=0.0)


In [21]:
pool._set_position("888", 90000, 95000, -200)
pool.show()

GlobalState(L=700.0, rP=63.245553203367585, tick=82900, fg_x=0.0, fg_y=0.0, proto_x=0.0, proto_y=0.0)
real reserves X=0.0 Y=0.0
---active ticks---
tick '80000': TickState(liq_net=300.0, liq_gross=300.0, f0_x=0.0, f0_y=0.0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
tick '90000': TickState(liq_net=-300.0, liq_gross=300.0, f0_x=0, f0_y=0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
tick '95000': TickState(liq_net=-400.0, liq_gross=400.0, f0_x=0, f0_y=0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
tick '70000': TickState(liq_net=400.0, liq_gross=400.0, f0_x=0.0, f0_y=0.0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
---positions---
poz '('123', 80000, 90000)': PositionState(liq=300, fr_x=0.0, fr_y=0.0)
poz '('888', 70000, 95000)': PositionState(liq=400, fr_x=0.0, fr_y=0.0)


-----------
-----------
2. deposit()

In [22]:
from v3_pool import *

pool= Pool("ETH", 18, "USDC", 6, bootstrap_rP=2000**0.5,tick_spacing=1)
pool.deposit("abc",x=2, y=4000, rPa=1333**0.5, rPb=3000**0.5)

x_dep=1.9979864402233023 y_dep=3999.217850013463 X returned 0.0020135597766977043 Y returned 0.7821499865372061


-----------
-----------
3. execute_swap_from_X(): breaking up trades

In [23]:
from v3_pool import *

pool1= Pool("ETH", 18, "USDC", 6, bootstrap_rP=2000**0.5,tick_spacing=1)
pool1.deposit("abc",x=2, y=4000, rPa=1333**0.5, rPb=3000**0.5)

pool1.execute_swap_from_X(3)
print()
pool1.show()

x_dep=1.9979864402233023 y_dep=3999.217850013463 X returned 0.0020135597766977043 Y returned 0.7821499865372061
Gap in liquidity... trying to get in range...
no more active ticks (liquidity) in this direction
swapped_dX=2.4493746062665265 swapped_dY=-3999.1778578349627 pool_X=4.447361046489829 _Y=0.03999217850014247

GlobalState(L=0.0, rP=36.509413246066714, tick=71954, fg_x=0.0, fg_y=0.0, proto_x=0.0, proto_y=0.0)
real reserves X=4.447361046489829 Y=0.03999217850014247
---active ticks---
tick '71955': TickState(liq_net=487.0, liq_gross=487.0, f0_x=0.0, f0_y=0.0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
tick '80067': TickState(liq_net=-487.0, liq_gross=487.0, f0_x=0, f0_y=0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
---positions---
poz '('abc', 71955, 80067)': PositionState(liq=487, fr_x=0.0, fr_y=0.0)


In [24]:
pool2= Pool("ETH", 18, "USDC", 6, bootstrap_rP=2000**0.5,tick_spacing=1)
pool2.deposit("abc",x=2, y=4000, rPa=1333**0.5, rPb=3000**0.5)
pool2.execute_swap_from_X(1)
pool2.execute_swap_from_X(1)
pool2.execute_swap_from_X(1)
print()
pool2.show()
assert pool2.X >= pool1.X, "breaking up trade should be slighlty more advantageous for pool"
# assert pool2.Y >= pool1.Y, "breaking down the trade should result in less Y going out"

x_dep=1.9979864402233023 y_dep=3999.217850013463 X returned 0.0020135597766977043 Y returned 0.7821499865372061
swapped_dX=1.0 swapped_dY=-1831.7681667411352 pool_X=2.9979864402233023 pool_Y=2167.4496832723275
swapped_dX=1.0 swapped_dY=-1547.5450858252436 pool_X=3.9979864402233023 pool_Y=619.9045974470839
Gap in liquidity... trying to get in range...
no more active ticks (liquidity) in this direction
swapped_dX=0.4493746062665272 swapped_dY=-619.8646052685842 pool_X=4.44736104648983 _Y=0.039992178499687725

GlobalState(L=0.0, rP=36.509413246066714, tick=71954, fg_x=0.0, fg_y=0.0, proto_x=0.0, proto_y=0.0)
real reserves X=4.44736104648983 Y=0.039992178499687725
---active ticks---
tick '71955': TickState(liq_net=487.0, liq_gross=487.0, f0_x=0.0, f0_y=0.0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
tick '80067': TickState(liq_net=-487.0, liq_gross=487.0, f0_x=0, f0_y=0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
---positions---
poz '('abc', 71955, 800

In [25]:
pool1.execute_swap_from_X(0.75)
pool2.execute_swap_from_X(0.75)

Gap in liquidity... trying to get in range...
no more active ticks (liquidity) in this direction
swapped_dX=0.0 swapped_dY=0.0 pool_X=4.447361046489829 _Y=0.03999217850014247
Gap in liquidity... trying to get in range...
no more active ticks (liquidity) in this direction
swapped_dX=0.0 swapped_dY=0.0 pool_X=4.44736104648983 _Y=0.039992178499687725


(0.0, 0.0)

----------
4. execute_swap_from_X(): pool starts with no liquidity in range

In [26]:
from v3_pool import *
pool3= Pool("SOL", 12, "USDC", 6, bootstrap_rP=150**0.5,tick_spacing=1)
pool3.deposit("dude",x=4, y=600, rPa=100**0.5, rPb=140**0.5)
pool3.show()
assert pool3.X == 0.0, "no X deposited as range below price"

x_dep=0.0 y_dep=598.9437669860457 X returned 4.0 Y returned 1.0562330139542837
GlobalState(L=0.0, rP=12.24744871391589, tick=50108, fg_x=0.0, fg_y=0.0, proto_x=0.0, proto_y=0.0)
real reserves X=0.0 Y=598.9437669860457
---active ticks---
tick '46054': TickState(liq_net=327.0, liq_gross=327.0, f0_x=0.0, f0_y=0.0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
tick '49418': TickState(liq_net=-327.0, liq_gross=327.0, f0_x=0.0, f0_y=0.0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
---positions---
poz '('dude', 46054, 49418)': PositionState(liq=327, fr_x=0.0, fr_y=0.0)


In [27]:
pool3.execute_swap_from_X(2)
pool3.show()

Gap in liquidity... trying to get in range...
swapped_dX=2.0 swapped_dY=-261.0792203432456 pool_X=2.0 pool_Y=337.86454664280006
GlobalState(L=327.0, rP=11.033214725935037, tick=48020, fg_x=0.0, fg_y=0.0, proto_x=0.0, proto_y=0.0)
real reserves X=2.0 Y=337.86454664280006
---active ticks---
tick '46054': TickState(liq_net=327.0, liq_gross=327.0, f0_x=0.0, f0_y=0.0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
tick '49418': TickState(liq_net=-327.0, liq_gross=327.0, f0_x=0.0, f0_y=0.0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
---positions---
poz '('dude', 46054, 49418)': PositionState(liq=327, fr_x=0.0, fr_y=0.0)


----------
5. execute_swap_from_X(): there is gap of liquidity on the way down while trying to fill whole amount

In [28]:
from v3_pool import *
pool4= Pool("SOL", 12, "USDC", 6, bootstrap_rP=200**0.5,tick_spacing=1)
pool4.deposit("dude",x=4, y=600, rPa=175**0.5, rPb=250**0.5)
pool4.deposit("mate",x=0, y=400, rPa=120**0.5, rPb=150**0.5)
# gap of liquidity between 175 and 150 
pool4.show()


x_dep=3.9932161139632054 y_dep=488.81423381261783 X returned 0.006783886036794584 Y returned 111.18576618738217
x_dep=0.0 y_dep=399.4265195744154 X returned 0.0 Y returned 0.5734804255845916
GlobalState(L=535.0, rP=14.142135623730951, tick=52985, fg_x=0.0, fg_y=0.0, proto_x=0.0, proto_y=0.0)
real reserves X=3.9932161139632054 Y=888.2407533870332
---active ticks---
tick '51650': TickState(liq_net=535.0, liq_gross=535.0, f0_x=0.0, f0_y=0.0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
tick '55217': TickState(liq_net=-535.0, liq_gross=535.0, f0_x=0, f0_y=0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
tick '47877': TickState(liq_net=309.0, liq_gross=309.0, f0_x=0.0, f0_y=0.0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
tick '50108': TickState(liq_net=-309.0, liq_gross=309.0, f0_x=0.0, f0_y=0.0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
---positions---
poz '('dude', 51650, 55217)': PositionState(liq=535, fr_x=0.0, fr_y

In [29]:
pool4.execute_swap_from_X(7)
pool4.show()

Gap in liquidity... trying to get in range...
Gap in liquidity... trying to get in range...
no more active ticks (liquidity) in this direction
swapped_dX=5.590203298887696 swapped_dY=-888.2318709794994 pool_X=9.583419412850901 _Y=0.008882407533860714
GlobalState(L=0.0, rP=10.954280744946413, tick=47876, fg_x=0.0, fg_y=0.0, proto_x=0.0, proto_y=0.0)
real reserves X=9.583419412850901 Y=0.008882407533860714
---active ticks---
tick '51650': TickState(liq_net=535.0, liq_gross=535.0, f0_x=0.0, f0_y=0.0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
tick '55217': TickState(liq_net=-535.0, liq_gross=535.0, f0_x=0, f0_y=0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
tick '47877': TickState(liq_net=309.0, liq_gross=309.0, f0_x=0.0, f0_y=0.0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
tick '50108': TickState(liq_net=-309.0, liq_gross=309.0, f0_x=0.0, f0_y=0.0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
---positions---
poz '('

-----------
-----------
6. execute_swap_from_Y(): breaking up trades

In [30]:
from v3_pool import *

pool5= Pool("ETH", 18, "USDC", 6, bootstrap_rP=2000**0.5,tick_spacing=1)
pool5.deposit("abc",x=2, y=4000, rPa=1333**0.5, rPb=3000**0.5)
pool5.show()
print()
pool5.execute_swap_from_Y(6000)
print()
pool5.show()

x_dep=1.9979864402233023 y_dep=3999.217850013463 X returned 0.0020135597766977043 Y returned 0.7821499865372061
GlobalState(L=487.0, rP=44.721359549995796, tick=76012, fg_x=0.0, fg_y=0.0, proto_x=0.0, proto_y=0.0)
real reserves X=1.9979864402233023 Y=3999.217850013463
---active ticks---
tick '71955': TickState(liq_net=487.0, liq_gross=487.0, f0_x=0.0, f0_y=0.0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
tick '80067': TickState(liq_net=-487.0, liq_gross=487.0, f0_x=0, f0_y=0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
---positions---
poz '('abc', 71955, 80067)': PositionState(liq=487, fr_x=0.0, fr_y=0.0)

Gap in liquidity... trying to get in range...
no more active ticks (liquidity) in this direction
swapped_dX=-1.9979664603589014 swapped_dY=4893.881200276067 pool_X=1.9979864400943015e-05 _Y=8893.09905028953

GlobalState(L=0.0, rP=54.77039692222591, tick=80067, fg_x=0.0, fg_y=0.0, proto_x=0.0, proto_y=0.0)
real reserves X=1.9979864400943015e-05 Y=88

In [31]:
pool6= Pool("ETH", 18, "USDC", 6, bootstrap_rP=2000**0.5,tick_spacing=1)
pool6.deposit("abc",x=2, y=4000, rPa=1333**0.5, rPb=3000**0.5)
pool6.execute_swap_from_Y(2000)
pool6.execute_swap_from_Y(2000)
pool6.execute_swap_from_Y(2000)
print()
pool6.show()
assert pool6.Y >= pool5.Y, "breaking up trade should be slighlty more advantageous for pool"
# assert pool6.X >= pool5.X, "breaking down the trade should result in less X going out"

x_dep=1.9979864402233023 y_dep=3999.217850013463 X returned 0.0020135597766977043 Y returned 0.7821499865372061
swapped_dX=-0.9158840833705683 swapped_dY=2000.0 pool_X=1.082102356852734 pool_Y=5999.217850013463
swapped_dX=-0.7737725429126204 swapped_dY=2000.0 pool_X=0.3083298139401135 pool_Y=7999.217850013463
Gap in liquidity... trying to get in range...
no more active ticks (liquidity) in this direction
swapped_dX=-0.3083098340757125 swapped_dY=893.8812002760709 pool_X=1.9979864400998526e-05 _Y=8893.099050289535

GlobalState(L=0.0, rP=54.77039692222591, tick=80067, fg_x=0.0, fg_y=0.0, proto_x=0.0, proto_y=0.0)
real reserves X=1.9979864400998526e-05 Y=8893.099050289535
---active ticks---
tick '71955': TickState(liq_net=487.0, liq_gross=487.0, f0_x=0.0, f0_y=0.0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
tick '80067': TickState(liq_net=-487.0, liq_gross=487.0, f0_x=0.0, f0_y=0.0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
---positions---
poz '('abc

In [32]:
pool5.execute_swap_from_Y(500)
pool6.execute_swap_from_Y(500)

Gap in liquidity... trying to get in range...
no more active ticks (liquidity) in this direction
swapped_dX=0.0 swapped_dY=0.0 pool_X=1.9979864400943015e-05 _Y=8893.09905028953
Gap in liquidity... trying to get in range...
no more active ticks (liquidity) in this direction
swapped_dX=0.0 swapped_dY=0.0 pool_X=1.9979864400998526e-05 _Y=8893.099050289535


(0.0, 0.0)

----------
7. execute_swap_from_Y(): pool starts with no liquidity in range

In [33]:
from v3_pool import *
pool7= Pool("SOL", 12, "USDC", 6, bootstrap_rP=150**0.5,tick_spacing=1)
pool7.deposit("dude",x=4, y=600, rPa=180**0.5, rPb=230**0.5)
pool7.show()
assert pool7.Y == 0.0, "no Y deposited as range below price"

x_dep=3.997362440038902 y_dep=0.0 X returned 0.002637559961097935 Y returned 600.0
GlobalState(L=0.0, rP=12.24744871391589, tick=50108, fg_x=0.0, fg_y=0.0, proto_x=0.0, proto_y=0.0)
real reserves X=3.997362440038902 Y=0.0
---active ticks---
tick '51932': TickState(liq_net=465.0, liq_gross=465.0, f0_x=0, f0_y=0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
tick '54383': TickState(liq_net=-465.0, liq_gross=465.0, f0_x=0, f0_y=0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
---positions---
poz '('dude', 51932, 54383)': PositionState(liq=465, fr_x=0.0, fr_y=0.0)


In [34]:
pool7.execute_swap_from_Y(200)
pool7.show()

Gap in liquidity... trying to get in range...
swapped_dX=-1.0766039259356373 swapped_dY=200.0 pool_X=2.920758514103265 pool_Y=200.0
GlobalState(L=465.0, rP=13.846404750028977, tick=52563, fg_x=0.0, fg_y=0.0, proto_x=0.0, proto_y=0.0)
real reserves X=2.920758514103265 Y=200.0
---active ticks---
tick '51932': TickState(liq_net=465.0, liq_gross=465.0, f0_x=0.0, f0_y=0.0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
tick '54383': TickState(liq_net=-465.0, liq_gross=465.0, f0_x=0, f0_y=0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
---positions---
poz '('dude', 51932, 54383)': PositionState(liq=465, fr_x=0.0, fr_y=0.0)


----------
8. execute_swap_from_Y(): there is gap of liquidity on the way up while trying to fill whole amount

In [35]:
from v3_pool import *
pool8= Pool("SOL", 12, "USDC", 6, bootstrap_rP=175**0.5,tick_spacing=1)
pool8.deposit("dude",x=2, y=600, rPa=150**0.5, rPb=190**0.5)
pool8.deposit("mate",x=3, y=0, rPa=210**0.5, rPb=250**0.5)
# gap of liquidity between 190 and 210 
pool8.show()


x_dep=1.8587444443840018 y_dep=599.9001781984224 X returned 0.14125555561599823 Y returned 0.09982180157760467
x_dep=2.9964567099858472 y_dep=0.0 X returned 0.003543290014152767 Y returned 0.0
GlobalState(L=611.0, rP=13.228756555322953, tick=51650, fg_x=0.0, fg_y=0.0, proto_x=0.0, proto_y=0.0)
real reserves X=4.8552011543698494 Y=599.9001781984224
---active ticks---
tick '50108': TickState(liq_net=611.0, liq_gross=611.0, f0_x=0.0, f0_y=0.0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
tick '52472': TickState(liq_net=-611.0, liq_gross=611.0, f0_x=0, f0_y=0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
tick '53473': TickState(liq_net=520.0, liq_gross=520.0, f0_x=0, f0_y=0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
tick '55217': TickState(liq_net=-520.0, liq_gross=520.0, f0_x=0, f0_y=0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
---positions---
poz '('dude', 50108, 52472)': PositionState(liq=611, fr_x=0.0, fr_y=0.0)


In [36]:
pool8.execute_swap_from_Y(1500)
pool8.show()

Gap in liquidity... trying to get in range...
Gap in liquidity... trying to get in range...
no more active ticks (liquidity) in this direction
swapped_dX=-4.855152602358306 swapped_dY=1025.4557302429805 pool_X=4.855201154363442e-05 _Y=1625.3559084414028
GlobalState(L=0.0, rP=15.811095915727572, tick=55217, fg_x=0.0, fg_y=0.0, proto_x=0.0, proto_y=0.0)
real reserves X=4.855201154363442e-05 Y=1625.3559084414028
---active ticks---
tick '50108': TickState(liq_net=611.0, liq_gross=611.0, f0_x=0.0, f0_y=0.0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
tick '52472': TickState(liq_net=-611.0, liq_gross=611.0, f0_x=0.0, f0_y=0.0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
tick '53473': TickState(liq_net=520.0, liq_gross=520.0, f0_x=0.0, f0_y=0.0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
tick '55217': TickState(liq_net=-520.0, liq_gross=520.0, f0_x=0.0, f0_y=0.0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
---positions--

-------------
-------------
9. withdraw()

In [37]:
from v3_pool import *
pool_w= Pool("SOL", 12, "USDC", 6, bootstrap_rP=200**0.5,tick_spacing=1)
pool_w.deposit("dude",x=4, y=600, rPa=175**0.5, rPb=250**0.5)
pool_w.show()

x_dep=3.9932161139632054 y_dep=488.81423381261783 X returned 0.006783886036794584 Y returned 111.18576618738217
GlobalState(L=535.0, rP=14.142135623730951, tick=52985, fg_x=0.0, fg_y=0.0, proto_x=0.0, proto_y=0.0)
real reserves X=3.9932161139632054 Y=488.81423381261783
---active ticks---
tick '51650': TickState(liq_net=535.0, liq_gross=535.0, f0_x=0.0, f0_y=0.0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
tick '55217': TickState(liq_net=-535.0, liq_gross=535.0, f0_x=0, f0_y=0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
---positions---
poz '('dude', 51650, 55217)': PositionState(liq=535, fr_x=0.0, fr_y=0.0)


In [38]:
pool_w.execute_swap_from_X(20)
# pool_w.execute_swap_from_Y(380)
pool_w.show()

Gap in liquidity... trying to get in range...
no more active ticks (liquidity) in this direction
swapped_dX=2.612879736633043 swapped_dY=-488.8093456702797 pool_X=6.606095850596248 _Y=0.004888142338131729
GlobalState(L=0.0, rP=13.228464158660637, tick=51649, fg_x=0.0, fg_y=0.0, proto_x=0.0, proto_y=0.0)
real reserves X=6.606095850596248 Y=0.004888142338131729
---active ticks---
tick '51650': TickState(liq_net=535.0, liq_gross=535.0, f0_x=0.0, f0_y=0.0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
tick '55217': TickState(liq_net=-535.0, liq_gross=535.0, f0_x=0, f0_y=0, s0_x=0.0, s0_y=0.0, i0_x=0.0, i0_y=0.0, sl0_x=0.0, sl0_y=0.0)
---positions---
poz '('dude', 51650, 55217)': PositionState(liq=535, fr_x=0.0, fr_y=0.0)


In [39]:

pool_w.withdraw("dude",liq=535, rPa=175**0.5, rPb=250**0.5)
pool_w.show()

x_out=6.606029789637744 y_out=0.0
GlobalState(L=0.0, rP=13.228464158660637, tick=51649, fg_x=0.0, fg_y=0.0, proto_x=0.0, proto_y=0.0)
real reserves X=6.606095850436589e-05 Y=0.004888142338131729
---active ticks---
---positions---
